In [2]:
using JuMP 
using Mosek

evaluarGraphLearningMean6Grande (generic function with 1 method)

In [ ]:
#En este script incluimos las funciones para recuperar poniendo una restriccion en la norma
#nuclear de la matriz a recuperar A. Lo hacemos para recuperacion lineal, l2 y espectral.

<h3>norma espectral<h3>

In [3]:
#Finds the matrix A which minimices the expected error of cluster structure. This
#time, A is a simmtric entrywise-positive matrix.
#The parameters are the same as in learnError function
function learnLowRankEspectral(weights,delta,B,n,beta)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
     @variable(m,S[1:N])
    @variable(m, W[1:n,1:n], Symmetric)
    @variable(m,A[1:n,1:n], Symmetric)
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
    C = @variable(m, [1:n, 1:n], Symmetric)
    @variable(m,W1[1:n,1:n])
    @variable(m,W2[1:n,1:n])
    #auxiliar variables to cumpute the nuclear norm of A
    @variable(m,A1[1:n,1:n])
    @variable(m,A2[1:n,1:n])
    
    #Constraints for Z
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constraints for C 
    @constraint(m,C.>=-(2*A-one*one'-W))
    @constraint(m,C.>=0)
    [@constraint(m, one'*C*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Constraints for W
    @SDconstraint(m,[W1 W;W W2]>=0)
    @constraint(m,trace(W1)+trace(W2)<=2*lambda)
    #constrains for A
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    @SDconstraint(m,[A1 A;A A2]>=0)
    
    #Objective function, calls the function prod
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n))+beta*0.5*(trace(A1)+trace(A2)))
    status = solve(m)
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

learnLowRankEspectral (generic function with 1 method)

<h3>norma l2<h3>

In [ ]:
#Finds the matrix A which minimices the expected error of cluster structure. This
#time, A is a simmtric entrywise-positive matrix.
#The parameters are the same as in learnError function
function learnLowRankl2(weights,delta,B,n,beta)
 N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    @variable(m,A[1:n,1:n], Symmetric)
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n], Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n])
    # u es una variable auxiliar para calcular la norma infinito de W.
   # @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
 
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for A.
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,vec(W)'vec(W)<=lambda)
     #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, one'*Q*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    #TT = STDOUT # save original STDOUT stream
    #redirect_stdout()
    solve(m)
    #redirect_stdout(TT) # restore STDOUT
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end

<h3>norma l1<h3>

In [ ]:
function learnLowRankLinear(weights,delta,B,n,beta)
    N = length(B)
    one =ones(n)
    m = Model(solver=MosekSolver())
    @variable(m,lambda>=0)
    @variable(m,S[1:N])
    @variable(m,A[1:n,1:n], Symmetric)
    #W debe ser simetrica? Creo que no.
    @variable(m, W[1:n,1:n],Symmetric)
    #Q es una variable auxiliar, juega el papel de Lambda en la demostracion
    @variable(m,Q[1:n,1:n],Symmetric)
    # u es una variable auxiliar para calcular la norma infinito de W.
    @variable(m,u>=0)
    #Z es un conjunto de matrices que permite calcular la norma 1 de A-B[i].
    Z = [@variable(m, [1:n, 1:n], Symmetric) for p in 1:N]
 
    #Constrains...
    #constrains for lambda already in the definition of the varible.
    #Constrains for A.
    @constraint(m,A.>=0)
    @constraint(m,A.<=1)
    #Constrains for Z.
    [@constraint(m, -Z[p].<= A-B[p]) for p in 1:N]
    [@constraint(m,  A-B[p].<=Z[p]) for p in 1:N]
    #Constrains for W. 
    @constraint(m,u<=lambda)
    @constraint(m,u .>=W)
    @constraint(m,u .>=-W)
     #Constrains for Q.
    @constraint(m,Q .>=-(2*A-one*one'-W))
    @constraint(m,Q .>=0)
    [@constraint(m, one'*Q*one<=S[p]-trace((2*A-one*one'-W)'*B[p])) for p in 1:N]
    #Objetivo
    @objective(m,Min,lambda*delta+ dot(weights,S+prod.(Z,n)))
    #TT = STDOUT # save original STDOUT stream
    #redirect_stdout()
    solve(m)
    #redirect_stdout(TT) # restore STDOUT
    return([getvalue(A),getvalue(S),getvalue(lambda),getvalue(W),getobjectivevalue(m)])
end